In [23]:
import io
from dataclasses import asdict, dataclass, field
import json
import time
from typing import Dict, Mapping, Optional, Set, Tuple
import os
import logging

import httpx
import polars as pl
from polars._typing import PolarsDataType

logger = logging.getLogger(__name__)

# -------------------------------------------------------------------------------------------------
# Configuration
# -------------------------------------------------------------------------------------------------

@dataclass
class Config:

    output_parquet: str = './data/naics_descriptions.parquet'

    url_codes: str = 'https://www.census.gov/naics/2022NAICS/2-6%20digit_2022_Codes.xlsx'
    url_index: str = 'https://www.census.gov/naics/2022NAICS/2022_NAICS_Index_File.xlsx'
    url_descriptions: str = 'https://www.census.gov/naics/2022NAICS/2022_NAICS_Descriptions.xlsx'
    url_exclusions: str = 'https://www.census.gov/naics/2022NAICS/2022_NAICS_Cross_References.xlsx'

    sheet_codes: str = 'tbl_2022_title_description_coun'
    sheet_index: str = '2022NAICS'
    sheet_descriptions: str = '2022_NAICS_Descriptions'
    sheet_exclusions: str = '2022_NAICS_Cross_References'
    schema_codes: Mapping[str, PolarsDataType] = field(
        default_factory=lambda: {
            'Seq. No.': pl.UInt32,
            '2022 NAICS US   Code': pl.Utf8,
            '2022 NAICS US Title': pl.Utf8,
        }
    )
    schema_index: Mapping[str, PolarsDataType] = field(
        default_factory=lambda: {'NAICS22': pl.Utf8, 'INDEX ITEM DESCRIPTION': pl.Utf8}
    )
    schema_descriptions: Mapping[str, PolarsDataType] = field(
        default_factory=lambda: {'Code': pl.Utf8, 'Description': pl.Utf8}
    )
    schema_exclusions: Mapping[str, PolarsDataType] = field(
        default_factory=lambda: {'Code': pl.Utf8, 'Cross-Reference': pl.Utf8}
    )

    rename_codes: Dict[str, str] = field(
        default_factory=lambda: {
            'Seq. No.': 'index',
            '2022 NAICS US   Code': 'code',
            '2022 NAICS US Title': 'title',
        }
    )
    rename_index: Dict[str, str] = field(
        default_factory=lambda: {'NAICS22': 'code', 'INDEX ITEM DESCRIPTION': 'examples_1'}
    )
    rename_descriptions: Dict[str, str] = field(
        default_factory=lambda: {'Code': 'code', 'Description': 'description'}
    )
    rename_exclusions: Dict[str, str] = field(
        default_factory=lambda: {'Code': 'code', 'Cross-Reference': 'excluded'}
    )


# -------------------------------------------------------------------------------------------------
# Import utilities
# -------------------------------------------------------------------------------------------------

def read_naics_xlsx(
    url: str, 
    sheet: str, 
    schema: Mapping[str, pl.DataType], 
    cols: Dict[str, str],
    max_retries: int = 3,
    initial_delay: float = 1.0,
    backoff_factor: float = 2.0,
    timeout: float = 30.0
) -> Optional[pl.DataFrame]:
    
    last_exception = None
    
    for attempt in range(max_retries + 1):

        try:
            resp = httpx.get(url, timeout=timeout)
            resp.raise_for_status()
            data = resp.content
            
            # Verify we got data
            if not data:
                raise ValueError(f"Empty response received from {url}")
            
            f_excel = io.BytesIO(data)
            
            return (
                pl
                .read_excel(
                    f_excel, 
                    sheet_name=sheet, 
                    columns=list(schema.keys()), 
                    schema_overrides=schema
                )
                .rename(mapping=cols)
            )
            
        except (httpx.HTTPError, httpx.TimeoutException, ValueError) as e:
            last_exception = e
            
            if attempt < max_retries:
                delay = initial_delay * (backoff_factor ** attempt)
                print(f"Attempt {attempt + 1}/{max_retries + 1} failed for {url}: {str(e)}")
                print(f"Retrying in {delay:.1f} seconds...")
                time.sleep(delay)

            else:
                print(f"All {max_retries + 1} attempts failed for {url}")
                raise last_exception
            

# -------------------------------------------------------------------------------------------------
# Download files
# -------------------------------------------------------------------------------------------------

def download_files(
        cfg: Config
) -> Tuple[
    Optional[pl.DataFrame], 
    Optional[pl.DataFrame], 
    Optional[pl.DataFrame], 
    Optional[pl.DataFrame]
]:
    
    # NAICS titles
    naics_titles = (
        read_naics_xlsx(
            url=cfg.url_codes, 
            sheet=cfg.sheet_codes, 
            schema=cfg.schema_codes, 
            cols=cfg.rename_codes
        )
    )

    # NAICS descriptions
    naics_descriptions = (
        read_naics_xlsx(
            url=cfg.url_descriptions,
            sheet=cfg.sheet_descriptions,
            schema=cfg.schema_descriptions,
            cols=cfg.rename_descriptions,
        )
    )

    # NAICS index file for examples
    naics_examples = (
        read_naics_xlsx(
            url=cfg.url_index, 
            sheet=cfg.sheet_index, 
            schema=cfg.schema_index, 
            cols=cfg.rename_index
        )
    )

    # NAICS cross reference file for exclusions
    naics_exclusions = (
        read_naics_xlsx(
            url=cfg.url_exclusions,
            sheet=cfg.sheet_exclusions,
            schema=cfg.schema_exclusions,
            cols=cfg.rename_exclusions,
        )
    )

    logger.info('Downloaded NAICS files successfully:')
    logger.info(f'  Titles observations: {naics_titles.height: ,}')
    logger.info(f'  Descriptions observations: {naics_descriptions.height: ,}')
    logger.info(f'  Examples observations: {naics_examples.height: ,}')
    logger.info(f'  Exclusions observations: {naics_exclusions.height: ,}')

    return naics_titles, naics_descriptions, naics_examples, naics_exclusions


# -------------------------------------------------------------------------------------------------
# Titles
# -------------------------------------------------------------------------------------------------

def get_titles(cfg: Config, titles_df: pl.DataFrame) -> Tuple[pl.DataFrame, Set[str]]:

    # Normalize combined sector codes (31-33, 44-45, 48-49), update index so 0-based
    naics_titles = (
        titles_df
        .with_columns(
            code=pl.when(pl.col('code').eq('31-33')).then(pl.lit('31', pl.Utf8))
                .when(pl.col('code').eq('44-45')).then(pl.lit('44', pl.Utf8))
                .when(pl.col('code').eq('48-49')).then(pl.lit('48', pl.Utf8))
                .otherwise(pl.col('code'))
        )
        .select(
            index=pl.col('index')
                    .sub(1),
            level=pl.col('code')
                    .str.len_chars()
                    .cast(pl.UInt8),
            code=pl.col('code'),
            title=pl.col('title'),
        )
    )

    # Create set of unique NAICS codes
    naics_codes = set(
        naics_titles
        .get_column('code')
        .unique()
        .sort()
        .to_list()
    )

    logger.info('Processed NAICS titles and created unique codes successfully:')
    logger.info(f'  Number of 2-6-digit NAICS codes: {len(naics_codes): ,}')
    logger.info(f'  Number of 2-6-digit NAICS titles: {naics_titles.height: ,}')

    return naics_titles, naics_codes

In [24]:
cfg = Config()

titles_df, descriptions_df, examples_df, exclusions_df = download_files(cfg)

naics_titles, naics_codes = get_titles(cfg, titles_df)

Attempt 1/4 failed for https://www.census.gov/naics/2022NAICS/2-6%20digit_2022_Codes.xlsx: The read operation timed out
Retrying in 1.0 seconds...
Attempt 1/4 failed for https://www.census.gov/naics/2022NAICS/2022_NAICS_Cross_References.xlsx: The read operation timed out
Retrying in 1.0 seconds...


In [7]:
naics_exclusions.head().to_dicts()

[{'code': '111110',
  'excluded': "Establishments engaged in growing soybeans in combination with grain(s) with the soybeans or grain(s) not accounting for one-half of the establishment's agricultural production (value of crops for market) are classified in U.S. Industry 111191, Oilseed and Grain Combination Farming."},
 {'code': '111120',
  'excluded': 'Growing soybeans--are classified in Industry 111110, Soybean Farming; and'},
 {'code': '111120',
  'excluded': "Growing oilseed(s) in combination with grain(s) with no one oilseed (or family of oilseeds) or grain(s) (or family of grains) accounting for one-half of the establishment's agricultural production (value of crops for market)--are classified in U.S. Industry 111191, Oilseed and Grain Combination Farming."},
 {'code': '111130',
  'excluded': 'Establishments primarily engaged in growing fresh green beans and peas are classified in U.S. Industry 111219, Other Vegetable (except Potato) and Melon Farming.'},
 {'code': '111140',
  '

In [12]:
# descriptions: normalize combined sector codes
naics_descriptions_1 = (
    naics_descriptions
    .with_columns(
        code=pl.when(pl.col('code').eq('31-33')).then(pl.lit('31', pl.Utf8))
                .when(pl.col('code').eq('44-45')).then(pl.lit('44', pl.Utf8))
                .when(pl.col('code').eq('48-49')).then(pl.lit('48', pl.Utf8))
                .otherwise(pl.col('code'))
    )
    .select('code', 'description')
)

# Split multiline descriptions and filter out section headers and cross-references
naics_descriptions_2 = (
    naics_descriptions_1
    .with_columns(
        description=pl.col('description')
                    .str.split('\r\n')
                    .list.eval(pl.element().filter(pl.element().str.len_chars() > 0))
    )
    .explode('description')
    .with_columns(
        description_id=pl.col('description')
                        .cum_count()
                        .over('code')
    )
    .select('code', 'description_id', 'description')
    .filter(
        (pl.col('description').ne('The Sector as a Whole')) &
        (~pl.col('description').str.contains('Cross-References.')) & 
        (pl.col('description').str.len_chars().gt(0))
    )
)

# Clean and normalize description text
naics_descriptions_3 = (
    naics_descriptions_2
    .select(
        code=pl.col('code')
            .str.strip_chars(),
        description_id=pl.col('description_id'),
        description=pl.col('description')
                        .str.strip_prefix(' ')
                        .str.strip_suffix(' ')
                        .str.replace_all(r'NULL', '')
                        .str.replace_all(r'See industry description for \d{6}\.', '')
                        .str.replace_all(r'<.*?>', '')
                        .str.replace_all(r'\xa0', ' ')
                        .str.replace_all('.', '. ', literal=True)
                        .str.replace_all('U. S. ', 'U.S.', literal=True)
                        .str.replace_all('e. g. ,', 'e.g.,', literal=True)
                        .str.replace_all('i. e. ,', 'i.e.,', literal=True)
                        .str.replace_all(';', '; ', literal=True)
                        .str.replace_all('31-33', '31', literal=True)
                        .str.replace_all('44-45', '44', literal=True)
                        .str.replace_all('48-49', '48', literal=True)
                        .str.replace_all(r'\s{2,}', ' '),
    )
    .filter(
        pl.col('description').ne('')
    )
)
naics_descriptions_3

code,description_id,description
str,u32,str
"""11""",2,"""The Agriculture, Forestry, Fis…"
"""11""",3,"""The establishments in this sec…"
"""11""",4,"""The sector distinguishes two b…"
"""11""",5,"""Excluded from the Agriculture,…"
"""111""",1,"""Industries in the Crop Product…"
…,…,…
"""928110""",7,"""Marine Corps"""
"""928110""",8,"""National Guard"""
"""928110""",9,"""Military courts"""


In [36]:
# Load descriptions and normalize combined sector codes
naics_exclusions_1 = (
    exclusions_df
    .filter(
        pl.col('excluded').str.contains(r' \d{2,6}'),
    )
    .with_columns(
        code=pl.when(pl.col('code').eq('31-33')).then(pl.lit('31', pl.Utf8))
                .when(pl.col('code').eq('44-45')).then(pl.lit('44', pl.Utf8))
                .when(pl.col('code').eq('48-49')).then(pl.lit('48', pl.Utf8))
                .otherwise(pl.col('code'))
    )
)
naics_exclusions_1

code,excluded
str,str
"""111110""","""Establishments engaged in grow…"
"""111120""","""Growing soybeans--are classifi…"
"""111120""","""Growing oilseed(s) in combinat…"
"""111130""","""Establishments primarily engag…"
"""111140""","""Establishments growing wheat i…"
…,…
"""928110""","""Regulating and administering w…"
"""928120""","""Establishments primarily engag…"
"""928120""","""Private establishments primari…"


In [37]:
naics_exclusions_2 = (
    naics_exclusions_1
    .group_by('code', maintain_order=True)
    .agg(
        excluded=pl.col('excluded')
    )
    .select(
        code=pl.col('code'), 
        description_id=pl.lit(1, pl.UInt32), 
        description=pl.col('excluded').list.join(' ')
    )
)
naics_exclusions_2

code,description_id,description
str,u32,str
"""111110""",1,"""Establishments engaged in grow…"
"""111120""",1,"""Growing soybeans--are classifi…"
"""111130""",1,"""Establishments primarily engag…"
"""111140""",1,"""Establishments growing wheat i…"
"""111150""",1,"""Growing sweet corn--are classi…"
…,…,…
"""926140""",1,"""Administering programs for dev…"
"""926150""",1,"""Government establishments prim…"
"""927110""",1,"""Private establishments primari…"


In [38]:

# Extract excluded activities (typically last description block for a code)
naics_exclusions_3 = (
    naics_descriptions_3
    .filter(
        pl.col('description_id').max().over('code').eq(pl.col('description_id')),
        pl.col('description').str.contains_any(['Excluded', 'excluded', 'exclude']),
        pl.col('description').str.contains(r' \d{2,6}'),
    )
    .select(
        code=pl.col('code')
            .str.strip_chars(),
        description_id=pl.col('description_id'),
        description=pl.col('description'),
    )
)
naics_exclusions_3

code,description_id,description
str,u32,str
"""11""",5,"""Excluded from the Agriculture,…"
"""22""",3,"""Excluded from this sector are …"
"""313""",4,"""Excluded from this subsector a…"
"""322""",5,"""Excluded from this subsector a…"
"""323""",5,"""Excluded from this subsector a…"
…,…,…
"""711""",4,"""Excluded from this subsector a…"
"""72""",3,"""Some establishments that provi…"
"""722""",3,"""Excluded from this subsector a…"


In [39]:
naics_exclusions_4 = (
    pl
    .concat([
        naics_exclusions_2, 
        naics_exclusions_3
    ])
    .filter(
        pl.col('description').is_not_null()
    )
    .with_columns(
        digit=pl.col('description')
                .str.extract_all(r' \d{2,6}')
                .list.eval(pl.element().str.strip_prefix(' '))
                .list.set_intersection(naics_codes)
                .list.drop_nulls()
    )
    .filter(
        pl.col('digit').list.len().gt(0)
    )
)
naics_exclusions_4

code,description_id,description,digit
str,u32,str,list[str]
"""111110""",1,"""Establishments engaged in grow…","[""111191""]"
"""111120""",1,"""Growing soybeans--are classifi…","[""111110"", ""111191""]"
"""111130""",1,"""Establishments primarily engag…","[""111219""]"
"""111140""",1,"""Establishments growing wheat i…","[""111191""]"
"""111150""",1,"""Growing sweet corn--are classi…","[""111219"", ""111191""]"
…,…,…,…
"""711""",4,"""Excluded from this subsector a…","[""7113"", ""722""]"
"""72""",3,"""Some establishments that provi…","[""81"", ""71"", ""51""]"
"""722""",3,"""Excluded from this subsector a…","[""487""]"


In [46]:
naics_exclusions_5 = (
    naics_exclusions_4
    .explode('digit')
    .filter(
        ~pl.col('digit').str.contains(pl.col('code'))
    )
    .select(
        level=pl.col('code')
                .str.len_chars()
                .cast(pl.UInt8),
        code=pl.col('code'),
        description=pl.col('description'),
        digit=pl.col('digit')
    )
    .sort('level', 'code')
)
naics_exclusions_5.tail().to_dicts()

[{'level': 6,
  'code': '928110',
  'description': 'Operating college-level military service academies--are classified in Industry 611310, Colleges, Universities, and Professional Schools; and Regulating and administering water transportation, such as the U.S. Coast Guard and the Merchant Marine--are classified in Industry 926120, Regulation and Administration of Transportation Programs.',
  'digit': '926120'},
 {'level': 6,
  'code': '928120',
  'description': 'Establishments primarily engaged in providing international trade financing, such as lending funds to foreign buyers of U.S. goods or lending funds to domestic buyers of imported goods, are classified in U.S. Industry 522299, International, Secondary Market, and All Other Nondepository Credit Intermediation; Private establishments primarily engaged in providing refugee settlement services are classified in Industry 624230, Emergency and Other Relief Services; Human rights organizations, peace advocacy organizations, and trade a